In [ ]:
# !pip install pandasql

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pandasql import sqldf
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
os.chdir("D:/Users/Desktop/Case 1/Case 1/Data/")

1.	Data– Using functions load the data. your team will use it for that purpose and needs to be easy.


In [ ]:
#Read Loans File
df_loans = pd.read_csv('2012_to_2014_loans_data.csv', header = 0, na_values = ['NA', 'NA ','NA  ', 'NA      ', 'NA    ','NA     ','NA   '])
print(df_loans.shape)
df_loans.tail(20)

In [ ]:
df_loans.dtypes

In [ ]:
df_loans['Respondent_ID'] = df_loans['Respondent_ID'].astype(str)
df_loans['Respondent_ID'] = df_loans['Respondent_ID'].str.lstrip('0').replace('', '0')

df_loans.head(10)

In [ ]:
# Read Institutions File
df_institutions = pd.read_csv('2012_to_2014_institutions_data.csv', header = 0, na_values = ['NA', 'NA ','NA  ', 'NA      ', 'NA    ','NA     ','NA   '])
print(df_institutions.shape)
df_institutions.head(10)

In [ ]:
df_institutions.dtypes

In [ ]:
df_institutions['Respondent_ID'] = df_institutions['Respondent_ID'].astype(str)
df_institutions['Respondent_ID'] = df_institutions['Respondent_ID'].str.lstrip('0')

df_institutions.head(10)

a. Merge the application data with institution data such that each loan application is assigned a “Respondent_Name”

In [ ]:
# Merge Datasets
new_df = df_loans.copy()
new_df = new_df.merge(df_institutions[['Respondent_ID', 'Agency_Code', 'As_of_Year', 'Respondent_Name_TS']], how = 'left', on = ['Respondent_ID', 'Agency_Code', 'As_of_Year'])
print(new_df.shape)
new_df.head(10)

In [ ]:
new_df[new_df['Respondent_Name_TS'].isnull()]

 Additionally, create a new attribute that buckets “Loan_Amount_000” into reasonable groups.


In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
new_df['Loan_Amount_000'].describe()

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (10,5))

axes[0].hist(new_df[new_df['Loan_Amount_000'] < 3000]['Loan_Amount_000'], bins=30, color='blue', alpha=0.7)
axes[0].set_title('Histogram')
axes[0].set_xlabel('Values')
axes[0].set_ylabel('Frequency')

axes[1].boxplot(new_df[new_df['Loan_Amount_000'] < 1000]['Loan_Amount_000'], vert=True, patch_artist=True) 
axes[1].set_title('Boxplot')
axes[1].set_xlabel('Data')
axes[1].set_ylabel('Values')

In [ ]:
plt.hist(new_df[new_df['Loan_Amount_000'] < 700]['Loan_Amount_000'], bins=30, color='lightblue', alpha=0.7)

In [ ]:
labels = ['Small', 'Medium', 'Large', 'Very Large']
new_df['Loan_Group'] = pd.qcut(new_df['Loan_Amount_000'], q=[0, 0.4, 0.65, 0.95, 1.0], labels=labels)
new_df['Loan_Group']

In [ ]:
new_df

i.	hmda_init() – Read the data files and return a pointer or object containing the expanded HMDA data from part a.


In [ ]:
def hmda_init():
    df_loans = pd.read_csv('2012_to_2014_loans_data.csv', header = 0, na_values = ['NA', 'NA ','NA  ', 'NA      ', 'NA    ','NA     ','NA   '])
    df_loans['Respondent_ID'] = df_loans['Respondent_ID'].astype(str)
    df_loans['Respondent_ID'] = df_loans['Respondent_ID'].str.lstrip('0').replace('', '0')
    
    df_institutions = pd.read_csv('2012_to_2014_institutions_data.csv', header = 0, na_values = ['NA', 'NA ','NA  ', 'NA      ', 'NA    ','NA     ','NA   '])
    df_institutions['Respondent_ID'] = df_institutions['Respondent_ID'].astype(str)
    df_institutions['Respondent_ID'] = df_institutions['Respondent_ID'].str.lstrip('0')

    new_df = df_loans.merge(df_institutions[['Respondent_ID', 'Agency_Code', 'As_of_Year', 'Respondent_Name_TS']], how = 'left', on = ['Respondent_ID', 'Agency_Code', 'As_of_Year'])
    
    labels = ['Small', 'Medium', 'Large', 'Very Large']
    new_df['Loan_Group'] = pd.qcut(new_df['Loan_Amount_000'], q=[0, 0.4, 0.65, 0.95, 1.0], labels=labels)
    
    return new_df

In [ ]:
df = hmda_init() 
df

In [ ]:
df.dtypes

ii. SQL – Write the following queries based on the tables provided.

1.	In the same view show the rank the agencies based on loan amount and rank by number of parent zip codes where they have provided loans

In [ ]:
run_query = lambda query: sqldf(query, globals())
q = """
    SELECT a.Agency_Code, SUM(a.Loan_Amount_000) AS Total_Loan_Amount_000, m.Unique_Zip_Count
    FROM df a
    JOIN (
          SELECT Agency_Code, COUNT(DISTINCT Parent_ZIP_Code) AS Unique_Zip_Count
          FROM df_institutions
          GROUP BY Agency_Code
          ) m
          ON a.Agency_Code = m.Agency_Code
    GROUP BY a.Agency_Code
    ORDER BY Unique_Zip_Count DESC;
    """

result_1 = run_query(q)
result_1

2.	Show the three largest loan amounts per county and the average of the county (repeated for each loan amount)


In [ ]:
q = """
    SELECT a.County_Name, a.Loan_Amount_000, a.rank, Avg_Loan_Amount_000
    FROM (
        SELECT County_Name, Loan_Amount_000, row_number() over (partition by County_Name order by Loan_Amount_000 desc) as rank 
        FROM df
        WHERE County_Name NOT IN ('None')
        ) a 
    LEFT JOIN (
               SELECT County_Name, AVG(Loan_Amount_000) AS Avg_Loan_Amount_000
               FROM df
               WHERE County_Name NOT IN ('None')
               GROUP BY County_Name) b
    ON a.County_Name = b.County_Name
    WHERE RANK <= 3;
    """
  
result_2 = run_query(q)
result_2

2.	Quality – wrong data is worse than no data at all
a.	Perform quality assessment on those Loan_Amount_000 and Respondent_Name. Do not stop at descriptive analytics; produce guidelines or techniques for quality check that can be applied.

* Loan_Amount_000

In [ ]:
df['Loan_Amount_000'].describe()

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(2, 2, 1)
plt.hist(df[df['Loan_Amount_000'] < 2000]['Loan_Amount_000'], bins=30, color='skyblue', edgecolor='black')
plt.title('Histogram')
plt.xlabel('Loan Amount ($000)')

plt.subplot(2, 2, 2)
plt.boxplot(df[df['Loan_Amount_000'] < 2000]['Loan_Amount_000'], vert=False)
plt.title('Box Plot of Loan Amounts')
plt.xlabel('Loan Amount ($000)')

plt.subplot(2, 2, 3)
sns.kdeplot(df[df['Loan_Amount_000'] < 2000]['Loan_Amount_000'], shade=True, color='green')
plt.title('Density Plot')
plt.xlabel('Loan Amount ($000)')

plt.subplot(2, 2, 4)
sns.violinplot(y=df[df['Loan_Amount_000'] < 2000]['Loan_Amount_000'], color='magenta')
plt.title('Violin Plot')
plt.xlabel('Loan Amount ($000)')

plt.tight_layout()
plt.show()

In [ ]:
def quality_check(df, column_name):
    
    negative_values = df[(df[column_name] < 0)][column_name]
    
    null_values = df[df[column_name].isnull()][column_name]
    
    not_numeric_values = df[~df[column_name].apply(lambda x: isinstance(x, (int, float)))][column_name]

    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[df[column_name] > upper_bound][column_name]
    
    null_sum = df[column_name].isnull().sum()
    completeness = (1-null_sum/df.shape[0])*100
    
    return{
        'negative_values': negative_values,
        'null_values': null_values,
        'not_numeric_values': not_numeric_values,
        'outliers': outliers,
        'completeness': completeness
    }

In [ ]:
check_loan = quality_check(df, 'Loan_Amount_000')
print("Negative Values", check_loan['negative_values'].shape[0])
print(check_loan['negative_values'])
print("\nNull Values", check_loan['null_values'].shape[0])
print(check_loan['null_values'])
print("\nNon-numeric Values", check_loan['not_numeric_values'].shape[0])
print(check_loan['not_numeric_values'])
print("\nOutliers", check_loan['outliers'].shape[0])
print(check_loan['outliers'])
print("\nCompleteness Percentage")
print(check_loan['completeness'])

In [ ]:
plt.hist(check_loan['outliers'], bins=30, color='skyblue', edgecolor='black')
plt.title('Histogram')
plt.xlabel('Outliers Loan Amount ($000)')

In [ ]:
#Copy df
df_copy_2 = df.copy()
df_copy_2 = df_copy_2.drop(list(check_loan['outliers'].index))
df_copy_2

In [ ]:
labels = ['Small', 'Medium', 'Large', 'Very Large']
df_copy_2['Loan_Group'] = pd.qcut(new_df['Loan_Amount_000'], q=[0, 0.25, 0.5, 0.75, 1.0], labels=labels)
df_copy_2['Loan_Group']

* Respondent Name

Al ser una variable de tipo carácter, revisaremos patrones en el nombre de las instituciones así como una limpieza de cualquier signo o carácter que no esté deacuerdo al formato establecido.

In [ ]:
def quality_check_2(df, column_name):
    df_copy = df.copy()

    null_values = df_copy[df_copy[column_name].isnull()][column_name]
    
    df_copy[column_name] = df_copy[column_name].apply(lambda x: re.sub(r'[^\w\s]', '', x))
    df_copy[column_name] = df_copy[column_name].str.upper().str.strip()

    df_copy['Name_Length'] = df_copy[column_name].str.len()
    anomalies = df_copy[(df_copy['Name_Length'] < 7)|
                        (df_copy['Name_Length'] > 35)][column_name]
    
    digit_values = df_copy[df_copy[column_name].astype(str).str.isdigit()][column_name]
    
    df_cleaned = df_copy.drop('Name_Length', axis = 1)
    
    return {
        'null_values': null_values, 
        'anomalies': anomalies,
        'cleaned_df': df_cleaned,
        'digit_values': digit_values
    }

In [ ]:
result_institutions = quality_check_2(df_copy_2, 'Respondent_Name_TS')
print("Null Values", result_institutions['null_values'].shape[0])
print(result_institutions['null_values'])
print("\nAnomalies", result_institutions['anomalies'].shape[0])
print(result_institutions['anomalies'].value_counts())
print("\nDigit Values", result_institutions['digit_values'].shape[0])
print(result_institutions['digit_values'])

In [ ]:
df_copy_2 = result_institutions['cleaned_df']
print(df_copy_2.shape)
df_copy_2.head()

Hemos revisado si nuestra columna tiene valores nulos, así como realizado un formato estándar para todos los datos y revisado aquellos nombres que pueden ser erróneos o solo contienen números. Para poder seguir con la revisión de los nombres sería importante revisar que estas instituciones se encontrarán en una base de nombres de instituciones financieras permitidas.

b. What other attributes (1 or 2) are relevant to keep regular quality check, why?


Parte primordial para la decisión de otorgar el préstamo se basa en el poder aqdquisitivo que el usuario tenga o en nuestro caso el 'Applicant_Income_000'. Es por eso que realizaremos un análisis para asegurarnos que la calidad de nuestros datos en esta columna sea la correcta.

* Applicant_Income_000

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(2, 2, 1)
plt.hist(df_copy_2[df_copy_2['Applicant_Income_000'] < 2000]['Applicant_Income_000'], bins=30, color='skyblue', edgecolor='black')
plt.title('Histogram')
plt.xlabel('Applicant Income ($000)')

plt.subplot(2, 2, 2)
plt.boxplot(df_copy_2[df_copy_2['Applicant_Income_000'] < 2000]['Applicant_Income_000'], vert=False)
plt.title('Box Plot of Loan Amounts')
plt.xlabel('Applicant Income ($000)')

plt.subplot(2, 2, 3)
sns.kdeplot(df_copy_2[df_copy_2['Applicant_Income_000'] < 2000]['Applicant_Income_000'], shade=True, color='green')
plt.title('Density Plot')
plt.xlabel('Applicant Income ($000)')

plt.subplot(2, 2, 4)
sns.violinplot(y=df_copy_2[df_copy_2['Applicant_Income_000'] < 2000]['Applicant_Income_000'], color='magenta')
plt.title('Violin Plot')
plt.xlabel('Applicant Income ($000)')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df_copy_2[df_copy_2['Applicant_Income_000'] < 600]['Applicant_Income_000'], bins=30, color='skyblue', edgecolor='black')
plt.title('Histogram')
plt.xlabel('Applicant Income ($000)')

In [ ]:
check_income = quality_check(df_copy_2, 'Applicant_Income_000')
print("Negative Values", check_income['negative_values'].shape[0])
print(check_income['negative_values'])
print("\nNull Values", check_income['null_values'].shape[0])
print(check_income['null_values'])
print("\nNon-numeric Values", check_income['not_numeric_values'].shape[0])
print(check_income['not_numeric_values'])
print("\nOutliers", check_income['outliers'].shape[0])
print(check_income['outliers'])
print("\nCompleteness Percentage")
print(check_income['completeness'])

In [ ]:
existing_indexes = set(df_copy_2.index)
indexes_to_drop = set(check_income['outliers'].index)
indexes_final = existing_indexes.intersection(indexes_to_drop)

df_copy_2 = df_copy_2.drop(list(indexes_final))
print(df_copy_2.shape)
df_copy_2.head(10)

In [ ]:
plt.figure(figsize = (8,6))
plt.hist(df_copy_2['Applicant_Income_000'], bins=30, color='skyblue', edgecolor='black')
plt.title('Histogram')
plt.xlabel('Applicant Income ($000)')
plt.show()

In [ ]:
median_income = df['Applicant_Income_000'].median()
df['Applicant_Income_000'] = df['Applicant_Income_000'].fillna(median_income)

Después de realizar el análisis de nuestra variable 'Applicant_Amount_000' se ha decidido imputar la mediana en aquellos valores faltantes.

* Uniqueness Primary Key

En el documento de 'Data Challenge - Metadata' Se menciona que si se combinan las variables 'As_of_Year', 'Agency_Code' y 'Respondent_ID' se puede considerar un identificador único para la institución en ese año. Es por esto que necesitamos tener en nuestra base de instituciones la seguridad que no existe duplicidad en el conjunto de estas variables.

In [ ]:
def quality_check_3(df, column_name):
    null_sum = df[column_name].isnull().sum()
    completeness = (1-null_sum/df.shape[0])*100
    return completeness

In [ ]:
print(quality_check_3(df_copy_2, 'As_of_Year'))
print(quality_check_3(df_copy_2, 'Agency_Code'))
print(quality_check_3(df_copy_2, 'Respondent_ID'))

In [ ]:
#Check for one digit
df_copy_2[~df_copy_2['Agency_Code'].astype(str).str.match(r'^\d$')].shape[0]

In [ ]:
print('Unique years in the dataset:',df['As_of_Year'].unique())

In [ ]:
df_copy_2['Identifier'] = df_copy_2['As_of_Year'].astype(str) + "_" + df_copy_2['Agency_Code'].astype(str) + "_" + df_copy_2['Respondent_ID'].astype(str)
df_copy_2['Identifier']

Hemos revisado que en nuestra base de instituciones, tengamos el $100\% $ de la información y de igual forma el atributo 'Agency_Code' sea un digito. Finalmente hemos creado una nueva columna que es la combinación de nuestros demás atributos.

3.	Create a visual narrative with data

El dataset solo contiene la información de 5 estados siendo West Virginia, Delaware, District of Columbia, Maryland y Virginia. Partiendo de esta información propondremos una hipótesis:

##### Hipótesis

Entrar en el mercado del estado de Maryland es la mejor opción actualmente para Keep Financial, pues se ha visto una tendencia positiva en los últimos años en los préstamos hipotecarios y unos ingresos estables durante este tiempo

In [ ]:
plt.figure(figsize=(12, 10))

df_copy_3 = df_copy_2.copy()
df_copy_3['As_of_Year'] = df_copy_3['As_of_Year'].astype(str)

plt.subplot(2, 2, 1)
sns.lineplot(x='As_of_Year', y='Loan_Amount_000', hue='State', data=df_copy_3, estimator = sum)
plt.title('Loan Amount Trends by State')
plt.xlabel('Year')
plt.ylabel('Total Loan Amount (000s)')
plt.legend(title='State')

plt.subplot(2, 2, 2)
sns.barplot(x='As_of_Year', y='Loan_Amount_000', hue='State', data=df_copy_3)
plt.title('Loan Amount Avg Trend by State')
plt.xlabel('Year')
plt.ylabel('Total Loan Amount (000s)')
plt.legend(title='State')

plt.subplot(2, 2, 3)
sns.lineplot(x='As_of_Year', y='Applicant_Income_000', hue='State', data=df_copy_3, estimator = sum)
plt.title('Applicant Income Trends by State')
plt.xlabel('Year')
plt.ylabel('Total Applicant Income (000s)')
plt.legend(title='State')

plt.subplot(2, 2, 4)
sns.barplot(x='As_of_Year', y='Applicant_Income_000', hue='State', data=df_copy_3)
plt.title('Applicant Income Avg Trend by State')
plt.xlabel('Year')
plt.ylabel('Total Applicant Income (000s)')
plt.legend(title='State')

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
sns.lineplot(x='As_of_Year', y='Loan_Amount_000', hue='State', data=df_copy_3[df_copy_3['State'].isin(['DC', 'DE', 'WV'])], estimator = sum)
plt.title('Loan Amount Trends by State')
plt.xlabel('Year')
plt.ylabel('Total Loan Amount (000s)')
plt.legend(title='State')

plt.subplot(1, 2, 2)
sns.lineplot(x='As_of_Year', y='Applicant_Income_000', hue='State', data=df_copy_3[df_copy_3['State'].isin(['DC', 'DE', 'WV'])], estimator = sum)
plt.title('Applicant Income Trends by State')
plt.xlabel('Year')
plt.ylabel('Total Applicant Income (000s)')
plt.legend(title='State')

plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
loans_per_state = df['State'].value_counts().reset_index()
loans_per_state.columns = ['State', 'Number_of_Loans']

plt.subplot(1, 2, 1)
sns.barplot(x='State', y='Number_of_Loans', data=loans_per_state)
plt.title('Number of Loans Submitted by State')
plt.xlabel('State')
plt.ylabel('Number of Loans')

plt.subplot(1, 2, 2)

plt.pie(x=loans_per_state['Number_of_Loans'], labels=loans_per_state['State'], autopct='%.0f%%')
plt.title('Percentage of Loans Submitted by State')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
ax1 = sns.countplot(
    x='As_of_Year',
    hue='State',
    data=df_copy_2
)
ax1.set_title('Number of Loans Submitted by Year and State')
ax1.set_xlabel('Year')
ax1.set_ylabel('Count of Loans')
plt.legend(loc='upper left', title='State')


df_agg = df_copy_3[['As_of_Year', 'State', 'Loan_Amount_000', 'Applicant_Income_000']].groupby(['As_of_Year', 'State']).sum()
df_agg.reset_index(inplace = True)
df_pivot = df_agg.pivot(index='State', columns='As_of_Year', values='Loan_Amount_000')
df_pivot['Ratio_2012_to_2014'] = 1 - df_pivot['2014'] / df_pivot['2012']

plt.subplot(1, 2, 2)
ax = df_pivot['Ratio_2012_to_2014'].plot(kind='bar', color='skyblue')
plt.title('Ratio of Loan Amounts: 2012 to 2014 by State')
plt.xlabel('State')
plt.ylabel('Ratio of Loan Amounts (2012/2014)')
plt.xticks(rotation=45)

for p in ax.patches:
    ax.annotate(format(p.get_height(), '.2f'),  
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center',
                va = 'center',
                xytext = (0, 9), 
                textcoords = 'offset points')

plt.tight_layout()
plt.show()

In [ ]:
g = sns.catplot(
    x='State', 
    kind='count', 
    hue='Loan_Group', 
    data=df_copy_2, 
    height=6, 
    aspect=2,
    legend = False
)

g.ax.set_title('Number of Loans by Loan Amount Category and State')
g.set_axis_labels('State', 'Count of Loans')
plt.legend(loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x ='Loan_Purpose_Description', data = df_copy_2, palette = 'Set1')
plt.ylabel('Count')
plt.xlabel('States')
plt.title('Number of Refinance and Purchase')

In [ ]:
g = sns.catplot(
    x='State', 
    kind='count', 
    hue='Loan_Purpose_Description', 
    data=df_copy_2, 
    height=6, 
    aspect=2,
    legend = False
)

g.ax.set_title('Number of Loans by State And Loan Purpose')
g.set_axis_labels('State', 'Count of Loans')
plt.legend(loc='upper left')
plt.show()

In [ ]:
respondent_per_state = df.groupby('State')['Respondent_Name_TS'].nunique().reset_index()
respondent_per_state_sorted = respondent_per_state.sort_values(by='Respondent_Name_TS', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=respondent_per_state_sorted, x='State', y='Respondent_Name_TS')
plt.title('Number of Different Respondents per State')
plt.xlabel('State Name')
plt.ylabel('Number of Unique Respondents')
plt.show()

Existe una tendencia a la baja en la cantidad solicitada de los préstamos, siendo Maryland el segundo posicionado en esta categoría. Al mismo tiempo en la cantidad promedio por préstamo y en los ingresos del aplicante están en segundo lugar, con un primer lugar de District of Columbia y una tercera posición para Virginia.

De la mano podemos ver que en el número de solucitudes, el orden se encuentra con Virgina, Maryland y después bastante lejos West Virginia, Delaware y District of Columbia. Virginia tiene números sólidos en cuanto la suma total de los préstamos, así como la demanda de estos, siendo el mercado más grande de estos estados.

Se puede apreciar que la mayoría de solicitudes de préstamos es para realizar refinanciamiento de la propiedad, esta característica se mantiene independientemente del estado. En cuanto a la cantidad de préstamo por categoría, la relación con los estados es muy similar, siendo el District of Columbia el único que su mayor porcentaje se encuentra en los préstamos de alto nivel, que lo pudimos ver repercutido en el préstamo promedio.

Después de realizar este análisis podemos concluír que Maryland es buena opción por el mercado que ofrece, sin embargo podemos encontrar mejores oportunidades dependiendo de las necesidades de la empresa, como lo puede ser el estado de Virginia. Faltan realizar nuevos análisis e incluír nuevas características de los préstamos para poder tomar una decisión.

4.	Basic stuff is done, now go a step further. What have piqued your mind so far? Have any worries about data quality? Make hypothesis, prototype a tool to improve this data product and highlight your area of expertise.

Empezaremos con la limpieza de aquellas variables 'Loan_Amount_000' y 'Applicant_Income_000', donde detectaremos la información que puede afectar nuestras decisiones y la eliminaremos para tener un análisis de mayor calidad.

In [ ]:
def get_outliers(df, column_name):
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    iqr = q3 - q1
    fence_low = q1 - 1.5 * iqr
    fence_high = q3 + 1.5 * iqr
    df_outliers = df.loc[(df[column_name] < fence_low) | (df[column_name] > fence_high)]
    return df_outliers, fence_low, fence_high

def plot_data(df, outliers, column_name):
    plt.figure(figsize=(10, 6))
    plt.scatter(df.index, df[column_name], label='Data')
    if not outliers.empty:
        plt.scatter(outliers.index, outliers[column_name], color='r', label='Outliers')
    plt.axhline(y=fence_low, color='g', linestyle='--', label='Lower Bound')
    plt.axhline(y=fence_high, color='g', linestyle='--', label='Upper Bound')
    plt.xlabel('Index')
    plt.ylabel(column_name)
    plt.legend()
    plt.grid(True)
    plt.show()
    
def remove_outliers(b):
    global df_copy_5
    global outliers
    df_copy_5 = df_copy_5.loc[(df_copy_5['Loan_Amount_000'] >= fence_low) & (df_copy_5['Loan_Amount_000'] <= fence_high)]
    outliers = pd.DataFrame(columns = df_copy_5.columns)
    plot_data(df_copy_5, outliers, 'Loan_Amount_000')

df_copy_5 = df.copy()
outliers, fence_low, fence_high = get_outliers(df_copy_5, 'Loan_Amount_000')
plot_data(df_copy_5, outliers, 'Loan_Amount_000')

button = widgets.Button(description="Remove Outliers")
button.on_click(remove_outliers)
display(button)

In [ ]:
def remove_outliers_2(b):
    global df_copy_5
    global outliers_2
    df_copy_5 = df_copy_5.loc[(df_copy_5['Applicant_Income_000'] >= fence_low) & (df_copy_5['Applicant_Income_000'] <= fence_high)]
    outliers_2 = pd.DataFrame(columns = df_copy_5.columns)
    plot_data(df_copy_5, outliers_2, 'Applicant_Income_000')

outliers_2, fence_low_2, fence_high_2 = get_outliers(df_copy_5, 'Applicant_Income_000')
plot_data(df_copy_5, outliers_2, 'Applicant_Income_000')

button = widgets.Button(description="Remove Outliers")
button.on_click(remove_outliers_2)
display(button)

In [ ]:
def update_plot(state):
    filtered_data = df_copy_2[df_copy_2['State'] == state]
    
    plt.figure(figsize=(10, 6))
    plt.hist(filtered_data['Loan_Amount_000'], bins=30, color='blue', alpha=0.7)
    plt.title(f'Loan Amount Distribution in {state}')
    plt.xlabel('Loan Amount (000s)')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

state_dropdown = widgets.Dropdown(
    options=['VA', 'MD', 'DC', 'WV', 'DE'],
    value='VA',
    description='State:',
    disabled=False,
)

widgets.interactive(update_plot, state=state_dropdown)

A continuación realizaremos la limpieza de las variables de tipo cadena, donde ajustaremos un estándar para los datos y sea más fácil la manipulación de estas variables.

In [ ]:
def clean_data(b):
    column_name = dropdown.value
    df_copy_5[column_name] = df_copy_5[column_name].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
    df_copy_5[column_name] = df_copy_5[column_name].str.upper().str.strip()

    with output:
        clear_output(wait=True)
        display(df_copy_5)

dropdown = widgets.Dropdown(
    options=df_copy_5.columns,
    description='Column:',
    disabled=False
)

button = widgets.Button(description="Clean Data")

output = widgets.Output()
button.on_click(clean_data)

display(dropdown, button, output)

Finalmente revisaremos aquellas variables que no tenemos todos los valores de estas, y decidiremos si eliminar aquellos registros que no contamos con la información completa.

In [ ]:
def plot_completeness(df):
    completeness = df.notnull().mean() * 100 
    plt.bar(completeness.index, completeness)
    plt.ylabel('Completeness (%)')
    plt.title('Column Completeness Before Cleaning')
    plt.xticks(rotation=90)
    plt.show()

def remove_nulls(b):
    selected_column = dropdown.value
    global df_copy_5
    df_copy_5 = df_copy_5[df_copy_5[selected_column].notnull()] 

    clear_output(wait=True)
    display(dropdown, button)
    plot_completeness(df_copy_5)

plot_completeness(df_copy_5)

dropdown = widgets.Dropdown(
    options=df_copy_5.columns,
    value=df_copy_5.columns[0],
    description='Column:',
    disabled=False,
)

button = widgets.Button(description="Remove Nulls")
button.on_click(remove_nulls)

display(dropdown, button)

### Virginia vs West Virginia

##### Hipótesis
Virginia mostrará una relación mayor a la relación de los ingresos de los prestatarios contra la cantidad solicitada en comparación con West Virginia, donde reflajará un mercado inmobiliario menos costoso en West Virginia y al mismo tiempo precios imobiliarios menores.

In [ ]:
plt.figure(figsize=(14, 7))
sns.histplot(data=df_copy_2[df_copy_2['State'].isin(['VA', 'WV'])], x='Loan_Amount_000', hue='State', element='step', stat='density', common_norm=False)
plt.title('Distribution of Loan Amounts in VA and WV')
plt.xlabel('Loan Amount (000s)')
plt.ylabel('Density')
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
sns.histplot(data=df_copy_2[df_copy_2['State'].isin(['VA', 'WV'])], x='Applicant_Income_000', hue='State', element='step', stat='density', common_norm=False)
plt.title('Distribution of Applicant Income in VA and WV')
plt.xlabel('Applicant Income (000s)')
plt.ylabel('Density')
plt.show()

In [ ]:
df_copy_4 = df_copy_2.copy()
df_copy_4['Loan_to_Income_Ratio'] = df_copy_4['Loan_Amount_000'] / df_copy_4['Applicant_Income_000']

plt.figure(figsize=(14, 7))
sns.histplot(data=df_copy_4[df_copy_4['State'].isin(['VA', 'WV'])  & (df_copy_4['Loan_to_Income_Ratio'] < 8)], x='Loan_to_Income_Ratio', hue='State', element='step', stat='density', common_norm=False)
plt.title('Distribution of Loan to Income Ratio in VA and WV')
plt.xlabel('Loan to Income Ratio')
plt.ylabel('Number of Loans')
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
sns.countplot(data=df_copy_2[df_copy_2['State'].isin(['VA', 'WV'])], x='Loan_Type_Description', hue='State')
plt.title('Loan Type in VA and WV')
plt.xlabel('Loan Type')
plt.ylabel('Number of Loans')
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
sns.countplot(data=df_copy_2[df_copy_2['State'].isin(['VA', 'WV'])], x='Lien_Status_Description', hue='State')
plt.title('Lien Status Distribution in VA and WV')
plt.ylabel('Number of Loans')
plt.xlabel('Lien Status')
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plt.pie(x=df_copy_2[df_copy_2['State'].isin(['VA'])]['Lien_Status_Description'].value_counts(normalize=True), labels=['First Lien', 'Subordinate Lien'], autopct='%.0f%%')
plt.title('Percentage Lien Status in Virginia')

plt.subplot(1, 2, 2)
plt.pie(x=df_copy_2[df_copy_2['State'].isin(['WV'])]['Lien_Status_Description'].value_counts(normalize=True), labels=['First Lien', 'Subordinate Lien'], autopct='%.0f%%')
plt.title('Percentage Lien Status in West Virginia')

plt.show()

In [ ]:
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plt.pie(x=df_copy_2[df_copy_2['State'].isin(['VA'])]['Loan_Type_Description'].value_counts(normalize=True), labels=df_copy_2[df_copy_2['State'].isin(['VA'])]['Loan_Type_Description'].value_counts(normalize=True).index, autopct='%.0f%%')
plt.title('Percentage Loan Type in Virginia')

plt.subplot(1, 2, 2)
plt.pie(x=df_copy_2[df_copy_2['State'].isin(['WV'])]['Loan_Type_Description'].value_counts(normalize=True), labels=df_copy_2[df_copy_2['State'].isin(['WV'])]['Loan_Type_Description'].value_counts(normalize=True).index, autopct='%.0f%%')
plt.title('Percentage Loan Type in West Virginia')

plt.show()

Nuestra investigación reveló que la relación entre el préstamo y los ingresos es diferente en Virginia y West Virginia, que valida nuestra hipótesis inicial que anticipaba diferencias significativas. El análisis puso de relieve factores como el mayor ingreso de solicitantes de ingresos medios y altos en Virginia y un mayor volumen de préstamos para veteranos en Virginia. Estos resultados sugieren que aunque la carga de deuda relativa es comparable, las características demográficas y económicas subyacentes varían y deben ser consideradas en nuestras estrategias de mercado. Por lo tanto, mientras que nuestra hipótesis se sostiene en términos de diferencias en la relación préstamo-ingreso, los datos subrayan oportunidades estratégicas específicas en cada estado, como enfocar los esfuerzos de marketing hacia productos especializados para veteranos en Virginia y desarrollar productos ajustados a prestatarios de ingresos más bajos en West Virginia.

### Modelo 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df_copy_2.dropna(inplace = True)
df_copy_model = df_copy_2.copy()
df_copy_model.drop(columns = ['Agency_Code', 'As_of_Year', 'Agency_Code_Description', 'Respondent_ID', 'Sequence_Number',
                              'State_Code', 'MSA_MD_Description', 'Census_Tract_Number', 'Tract_to_MSA_MD_Income_Pct', 
                              'FFIEC_Median_Family_Income', 'Number_of_Owner_Occupied_Units', 'County_Name', 'County_Code',
                              'Conforming_Limit_000', 'Conventional_Conforming_Flag', 'Identifier', 
                              'As_of_Year', 'Respondent_Name_TS', 'FFIEC_Median_Family_Income', 'MSA_MD',
                              'Loan_Group','Conventional_Status'], inplace = True, axis = 0)
df_copy_model.reset_index(drop=True, inplace=True)
df_copy_model.columns

In [ ]:
encoder = OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(df_copy_model[['Loan_Purpose_Description', 'Lien_Status_Description', 
                                                        'Loan_Type_Description', 'State', 
                                                        'Conforming_Status']])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out())
X = pd.concat([encoded_df, df_copy_model[['Applicant_Income_000']]], axis=1)
y = df_copy_model['Loan_Amount_000']

In [ ]:
X.isnull().sum()

In [ ]:
corr_matrix = X.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, linewidths=.5)
plt.title('Matriz de Correlación de Variables')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
importances = model.coef_
features = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': abs(importances)
})

features = features.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(features['Feature'], features['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Feature Importances')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
print("Coeficientes del Modelo:")
print(pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': model.coef_
}))

In [ ]:
features = ['Loan_Purpose_Description_Purchase', 'Loan_Purpose_Description_Refinance',
            'Lien_Status_Description_First Lien', 'Lien_Status_Description_Subordinate Lien',
            'Loan_Type_Description_Conventional', 'Loan_Type_Description_FHA insured',
            'Loan_Type_Description_FSA/RHS guaranteed', 'Loan_Type_Description_VA guaranteed',
            'State_DC', 'State_DE', 'State_MD', 'State_VA', 'State_WV',
            'Conforming_Status_Conforming', 'Conforming_Status_Jumbo', 'Applicant_Income_000']
df_scenarios = pd.DataFrame(columns=features)
df_scenarios.loc['West Virginia'] = [0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 50]
df_scenarios.loc['Virginia'] = [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 50]

predictions = model.predict(df_scenarios)

print("Predicciones de préstamo:")
print("West Virginia: ${:,.2f}".format(predictions[0]))
print("Virginia: ${:,.2f}".format(predictions[1]))